Claudio Palmeri 2062671 02/22/2023


My project will be focused around the EMNIST dataset, a dataset similar to the MNIST dataset. This is a classification tasks where we try to differentiate between handwritten letters (in contrast with the MNIST dataset that contains handwritten digits).


First of all I have to import all the relevant libraries and the files that professor Alberto Testolin used during our course.

In [ ]:
def _get_files_from_repo(files, repo):
  repository_url = f"https://raw.githubusercontent.com/flavio2018/{repo}/master/"
  for file in files:
    ! wget -O {file} {repository_url}{file}

In [ ]:
%%capture
files = ["DBN.py", "RBM.py"]
_get_files_from_repo(files, "Deep-Belief-Network-pytorch")

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import scipy.cluster as cluster
import sklearn.preprocessing
import torch
import torch.nn.functional as functional
import torchvision as tv

from DBN import DBN
from tqdm.notebook import tqdm

RuntimeError: ignored

The most efficient (and thus fastest) way to handle the processing of our data is by using a GPU, with the following instruction we check if this is possible in the current eviroment.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


We proceed now to download the EMNIST dataset that consist in 145600 28x28 gray scale images.

In order to be able to compute the accuracy of our models we need to separate our dataset in a training and a test set. We don't need to add additional instructions in order to obtain this separation since the developers of the library torch added the argument "train".

This boolean variable decides whether we are considering the training set (train=True, 124800 images) or the test set (train=False, 20800 images).

Since we are using a gray scale, every value in our datasets will be an integer included in the interval [0,255].  
In order to get better perfomances in our models I decided to divide each value by 255, this way we will only have values belonging to the interval [0,1].

In [ ]:
emnist_train=tv.datasets.EMNIST('data/', train=True ,download=True,split="letters")
emnist_test=tv.datasets.EMNIST('data/', train=False ,download=True,split="letters")
emnist_train.data = (emnist_train.data.type(torch.FloatTensor)/255)
emnist_test.data = (emnist_test.data.type(torch.FloatTensor)/255)

Before creating our models, i decided to print some examples of the images.

By doing this I observed 2 things:  
  
* In order to plot the images with the correct orientation (i.e. the one us humans are more familiar with) I was forced to both plot the transpose of the original image and also to change the orientation of the y-axis.

* Since our classes are expressed with a number between 1 and 26, I created an array containing the letters of the alphabet. This way we don't have to manually count the letters to understand which class the current image belongs to.

In [ ]:
alfa=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]

for i in range(5):
  print("The letter shown is the letter: {}".format(  alfa[emnist_train.targets[i]-1]   ) )
  plt.imshow(emnist_train.data[i].T, cmap = 'gray')
  plt.axis([0,27,27,0])
  plt.show()

We then transfer our datasets to the GPU device in order to make our training process quicker.

In [ ]:
emnist_train.data = emnist_train.data.to(device)
emnist_test.data = emnist_test.data.to(device)
emnist_train.targets = emnist_train.targets.to(device)
emnist_test.targets = emnist_test.targets.to(device)

We can begin now to train some models on our dataset.
The first one we will consider is a deep belief network (DBN). This is an unsupervised deep learning model and thus by itself it can't be used for our classification task.

A DBN is simply a stack of restricted Boltzman machines (RBM) that can be trained with contrastive divergence either together or one by one (the parameter k is the number of RBM trained at the same time).
We want to minimize the difference between the empirical data and the patterns generated by the network.

This is computationally expensive but can be approximated by using the distortions of the original data: We basically insert our data at the bottom of our DBN, we compute the neuron activations by going up until the last level and then we go in the opposite direction. Ideally we want to reconstruct the original input and we this doesn't happen we adjust the weights of our network accordingly.

In [ ]:
dbn_emnist = DBN(visible_units=28*28,
                hidden_units=[400, 600, 800, 1000],
                k=1,
                learning_rate=0.1,
                learning_rate_decay=False,
                initial_momentum=0.5,
                final_momentum=0.95,
                weight_decay=0.0001,
                xavier_init=False,
                increase_to_cd_k=False,
                use_gpu=torch.cuda.is_available())

In [ ]:
num_epochs = 50
batch_size = 125

dbn_emnist.train_static(
    emnist_train.data,
    emnist_train.targets,
    num_epochs,
    batch_size
)

After having trained our DBN, our hope is that the neurons in the deepest layers have picked up some of the characteristics of our datasets.  

In order to check that we can visualize the weights of our models that are called receptive fields.

Before doing that we need to:
* apply threshold to reduce noise
* apply a min max scaler to fairly compare different receptive fields
* eventually projecting them in a lower dimensional space

In [ ]:
def get_weights(dbn, layer):
  return dbn.rbm_layers[layer].W.cpu().numpy()

def apply_threshold(weights, threshold=0):
  return weights * (abs(weights) > threshold)

def plot_layer_receptive_fields(weights):
  num_subplots = 100
  n_rows_cols = int(math.sqrt(num_subplots))
  fig, axes = plt.subplots(n_rows_cols, n_rows_cols, sharex=True, sharey=True, figsize=(10, 10))
  for i in range(num_subplots):
    row = i % n_rows_cols
    col = i // n_rows_cols
    axes[row, col].imshow(weights[i,:].reshape((28,28)), cmap=plt.cm.gray)  # here we select the weights we want to plot

def apply_min_max_scaler(learned_weights):
  original_shape = learned_weights.shape
  min_max_scaler = sklearn.preprocessing.MinMaxScaler()
  min_max_scaled_learned_weights = min_max_scaler.fit_transform(learned_weights.ravel().reshape(-1,1))
  min_max_scaled_learned_weights = min_max_scaled_learned_weights.reshape(original_shape)
  return min_max_scaled_learned_weights

In [ ]:
#learned_weights_layer_1 = get_weights(dbn_emnist, layer=0)
#learned_weights_layer_1 = apply_threshold(learned_weights_layer_1, .1)
#learned_weights_layer_1 = apply_min_max_scaler(learned_weights_layer_1)

#plot_layer_receptive_fields(learned_weights_layer_1.T)

In [ ]:
learned_weights_layer_1 = get_weights(dbn_emnist, layer=0)
learned_weights_layer_2 = get_weights(dbn_emnist, layer=1)
learned_weights_layer_3 = get_weights(dbn_emnist, layer=2)
learned_weights_layer_4 = get_weights(dbn_emnist, layer=3)

learned_weights_layer_1 = apply_threshold(learned_weights_layer_1, 0.1)
learned_weights_layer_2 = apply_threshold(learned_weights_layer_2, 0.1)
learned_weights_layer_3 = apply_threshold(learned_weights_layer_3, 0.1)
learned_weights_layer_4 = apply_threshold(learned_weights_layer_4, 0.1)

learned_weights_layer_1 = apply_min_max_scaler(learned_weights_layer_1)
learned_weights_12_product = (learned_weights_layer_1 @ learned_weights_layer_2)  # here we do the projection (matrix multiplication)
learned_weights_23_product = (learned_weights_12_product @ learned_weights_layer_3)  # here we do the projection
learned_weights_34_product = (learned_weights_23_product @ learned_weights_layer_4)
learned_weights_12_product = apply_threshold(learned_weights_12_product, 0.1)
learned_weights_12_product = apply_min_max_scaler(learned_weights_12_product)
learned_weights_23_product = apply_threshold(learned_weights_23_product, 0.1)
learned_weights_23_product = apply_min_max_scaler(learned_weights_23_product)
learned_weights_34_product = apply_threshold(learned_weights_34_product, 0.1)
learned_weights_34_product = apply_min_max_scaler(learned_weights_34_product)

plot_layer_receptive_fields(learned_weights_layer_1.T)



In [ ]:
plot_layer_receptive_fields(learned_weights_12_product.T)

In [ ]:
plot_layer_receptive_fields(learned_weights_23_product.T)

In [ ]:
plot_layer_receptive_fields(learned_weights_34_product.T)

In [ ]:
#learned_weights_layer_1 = get_weights(dbn_emnist, layer=0)
#learned_weights_layer_2 = get_weights(dbn_emnist, layer=1)

###learned_weights_layer_1 = apply_threshold(learned_weights_layer_1, 0.1)
#learned_weights_layer_2 = apply_threshold(learned_weights_layer_2, 0.1)

#learned_weights_product = (learned_weights_layer_1 @ learned_weights_layer_2)  # here we do the projection
#learned_weights_product = apply_threshold(learned_weights_product, 0.1)
#learned_weights_product = apply_min_max_scaler(learned_weights_product)

#plot_layer_receptive_fields(learned_weights_product.T)

In [ ]:
#learned_weights_layer_1 = get_weights(dbn_emnist, layer=0)
#learned_weights_layer_2 = get_weights(dbn_emnist, layer=1)
#learned_weights_layer_3 = get_weights(dbn_emnist, layer=2)

#learned_weights_layer_1 = apply_threshold(learned_weights_layer_1, 0.1)
#learned_weights_layer_2 = apply_threshold(learned_weights_layer_2, 0.1)
#learned_weights_layer_3 = apply_threshold(learned_weights_layer_3, 0.1)

#learned_weights_12_product = (learned_weights_layer_1 @ learned_weights_layer_2)  # here we do the projection
#learned_weights_23_product = (learned_weights_12_product @ learned_weights_layer_3)  # here we do the projection
#learned_weights_23_product = apply_threshold(learned_weights_23_product, 0.1)
#learned_weights_23_product = apply_min_max_scaler(learned_weights_23_product)

#plot_layer_receptive_fields(learned_weights_23_product.T)

Conclusion:

By using a clustering algorithm and computing the mean of the representation learned by each class we can also see which classes are closer from an internal representation standpoint.

If the DBN is behaving as predicted, similar letters will have a similar internal representation and thus will be closer to each other.

We can visualize this by using a dendogram.

In [ ]:
def get_kth_layer_repr(input, k, device):
  flattened_input = input.view((input.shape[0], -1)).type(torch.FloatTensor).to(device)
  hidden_repr, __ = dbn_emnist.rbm_layers[k].to_hidden(flattened_input)  # here we access the RBM object
  return hidden_repr

In [ ]:
hidden_repr_layer_1 = get_kth_layer_repr(emnist_train.data, 0, device)
hidden_repr_layer_2 = get_kth_layer_repr(hidden_repr_layer_1, 1, device)
hidden_repr_layer_3 = get_kth_layer_repr(hidden_repr_layer_2, 2, device)
hidden_repr_layer_4 = get_kth_layer_repr(hidden_repr_layer_3, 3, device)

In [ ]:
def get_mask(label):  # we use this function to filter by class
  labels = emnist_train.targets.cpu().numpy()
  return labels == label

def get_label_to_mean_hidd_repr(hidden_representation):
  hidden_representation_np = hidden_representation.cpu().numpy()
  return {
    label: hidden_representation_np[get_mask(label)].mean(axis=0)  # here we filter by class and compute the mean
    for label in range(1,27)
  }

def get_hidden_reprs_matrix(hidden_representation):  # we use this to build the matrices
  label_to_mean_hidd_repr = get_label_to_mean_hidd_repr(hidden_representation)
  return np.concatenate(
    [np.expand_dims(label_to_mean_hidd_repr[label], axis=0)  # here we adjust the shape of centroids to do the concat
    for label in range(1,27)])

In [ ]:
mean_hidd_repr_matrix_4 = get_hidden_reprs_matrix(hidden_repr_layer_4)

In [ ]:
def plot_dendrogram(mean_repr_matrix, title=""):
  fig, ax = plt.subplots()
  linkage = cluster.hierarchy.linkage(mean_repr_matrix, method="complete")  # we run the clustering algorithm here
  dendrogram = cluster.hierarchy.dendrogram(linkage)
  ax.set_title(title)

In [ ]:
plot_dendrogram(mean_hidd_repr_matrix_4, "Fourth hidden layer")

As a said earlier, a DBN network can be used to make prediction because it is a unsupervised learning tecnique.
We can attach at each different layer a linear classifier and train it on the inputs given by the respective layer.

If we see an improvement in the performances by using deeper layers it will mean that they managed to organize the information better.

In [ ]:
class LinearModel(torch.nn.Module):
  def __init__(self, layer_size):
    super().__init__()
    self.linear = torch.nn.Linear(layer_size, 27)

  def forward(self, x):
    return self.linear(x)

In [ ]:
layer_size = dbn_emnist.rbm_layers[0].W.shape[1]
linear1 = LinearModel(layer_size).to(device)

layer_size = dbn_emnist.rbm_layers[1].W.shape[1]
linear2 = LinearModel(layer_size).to(device)

layer_size = dbn_emnist.rbm_layers[2].W.shape[1]
linear3 = LinearModel(layer_size).to(device)

layer_size = dbn_emnist.rbm_layers[3].W.shape[1]
linear4 = LinearModel(layer_size).to(device)

In [ ]:
def train(network, input, epochs=1500):
  optimizer = torch.optim.SGD(network.parameters(), lr=0.05)
  loss_fn = torch.nn.CrossEntropyLoss()

  for epoch in range(epochs):
    optimizer.zero_grad()
    predictions = network(input).squeeze()
    targets = emnist_train.targets.reshape(predictions.shape[0])  # here are the labels
    loss = loss_fn(predictions, targets)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
      print("epoch : {:3d}/{}, loss = {:.4f}".format(epoch + 1, epochs, loss))

In [ ]:
train(linear1, hidden_repr_layer_1)

In [ ]:
train(linear2, hidden_repr_layer_2)

In [ ]:
train(linear3, hidden_repr_layer_3)

In [ ]:
train(linear4, hidden_repr_layer_4)

In [ ]:
hidden_repr_layer_1_test = get_kth_layer_repr(emnist_test.data, 0, device)
hidden_repr_layer_2_test = get_kth_layer_repr(hidden_repr_layer_1_test, 1, device)
hidden_repr_layer_3_test = get_kth_layer_repr(hidden_repr_layer_2_test, 2, device)
hidden_repr_layer_4_test = get_kth_layer_repr(hidden_repr_layer_3_test, 3, device)

In [ ]:
predictions_test1 = linear1(hidden_repr_layer_1_test)
predictions_test2 = linear2(hidden_repr_layer_2_test)
predictions_test3 = linear3(hidden_repr_layer_3_test)
predictions_test4 = linear4(hidden_repr_layer_4_test)

In [ ]:
def compute_accuracy(predictions_test, targets):
  predictions_indices = predictions_test.max(axis=1).indices  # convert probabilities to indices
  accuracy = (predictions_indices == targets).sum() / len(targets)
  return accuracy.item()

In [ ]:
compute_accuracy(predictions_test1, emnist_test.targets)

In [ ]:
compute_accuracy(predictions_test2, emnist_test.targets)

In [ ]:
compute_accuracy(predictions_test3, emnist_test.targets)

In [ ]:
compute_accuracy(predictions_test4, emnist_test.targets)

As we can see, there is a improvement in the performance of our linear read out.
The more the original image is processed by the DBN the easier it becomes for him to do properly classify our data.

In order to compare these results, we will train now a deep feed forward network. This is a supervised learning tecnique that uses layers of connected formal neurons whose weights are optimized by using backpropagation.

Backpropagation consists in trying to minimize a loss function (a measure on how wrong we are at predicting) by using gradient descent.

In order to keep it fair, we want the number of epochs of training here to be equal to the sum of the epochs needed to both train the DBN and the linear read out.

In [ ]:
class Feedforward(torch.nn.Module):
  def __init__(self, first_hidden_layer_size, second_hidden_layer_size, third_hidden_layer_size):
    super().__init__()
    self.first_hidden = torch.nn.Linear(784, first_hidden_layer_size)
    self.second_hidden = torch.nn.Linear(first_hidden_layer_size, second_hidden_layer_size)
    self.third_hidden = torch.nn.Linear(second_hidden_layer_size, third_hidden_layer_size)
    self.output = torch.nn.Linear(third_hidden_layer_size, 27)

  def forward(self, input):
    relu = torch.nn.ReLU()
    first_hidden_repr = relu(self.first_hidden(input))
    second_hidden_repr = relu(self.second_hidden(first_hidden_repr))
    third_hidden_repr = relu(self.third_hidden(second_hidden_repr))
    output = self.output(third_hidden_repr)
    return output

In [ ]:
ffnn = Feedforward(400, 500, 800).to(device)

In [ ]:
train(ffnn, emnist_train.data.reshape((124800, 784)), epochs=1500)

In [ ]:
predictions_ffnn = ffnn(emnist_test.data.reshape((20800, 784)))

In [ ]:
compute_accuracy(predictions_ffnn, emnist_test.targets)

Conclusion:

Now we want to the the robustness of our models against noise (small random perturpations of our original input).

In [ ]:
def inject_noise(emnist_data, noise_level):
  random_gaussian_tensor = torch.randn(emnist_data.shape, device=device)*noise_level
  return emnist_data+random_gaussian_tensor
  ### TASK: create a very simple function that adds some Gaussian noise (see torch.randn function) to the MNIST data

In [ ]:
noise_level = 0.3
emnist_test_with_noise = inject_noise(emnist_test.data, noise_level)
__ = plt.imshow(emnist_test_with_noise[1].reshape(28, 28).to("cpu"), cmap="gray")

After having defined a function capable of adding noise and checking that we got the desired results, we can compute the accuracy of our different models with different level of noise.

In [ ]:
def get_accuracy_values_at_noise_level(noise_level):

  mnist_test_with_noise = inject_noise(emnist_test.data, noise_level)  # first, let's create noisy test images

  hidden_repr_layer_1_noisy = get_kth_layer_repr(mnist_test_with_noise, 0, device)  # here we compute the DBN representations
  hidden_repr_layer_2_noisy = get_kth_layer_repr(hidden_repr_layer_1_noisy, 1, device)
  hidden_repr_layer_3_noisy = get_kth_layer_repr(hidden_repr_layer_2_noisy, 2, device)
  hidden_repr_layer_4_noisy = get_kth_layer_repr(hidden_repr_layer_3_noisy, 3, device)

  predictions_first_hidden_noisy = linear1(hidden_repr_layer_1_noisy)  # here we use the previously-trained read-out classifiers
  predictions_second_hidden_noisy = linear2(hidden_repr_layer_2_noisy)
  predictions_third_hidden_noisy = linear3(hidden_repr_layer_3_noisy)
  predictions_fourth_hidden_noisy = linear4(hidden_repr_layer_4_noisy)

  accuracy_first_hidden = compute_accuracy(predictions_first_hidden_noisy, emnist_test.targets)
  accuracy_second_hidden = compute_accuracy(predictions_second_hidden_noisy, emnist_test.targets)
  accuracy_third_hidden = compute_accuracy(predictions_third_hidden_noisy, emnist_test.targets)
  accuracy_fourth_hidden = compute_accuracy(predictions_fourth_hidden_noisy, emnist_test.targets)

  ### TASK: repeat a similar process for the feed-forward model (NB: make sure you reshape the input data appropriately!)
  predictions_ffnn_noisy = ffnn(emnist_test_with_noise.reshape((20800, 784)))
  accuracy_ffnn=compute_accuracy(predictions_ffnn_noisy, emnist_test.targets)

  return accuracy_first_hidden, accuracy_second_hidden, accuracy_third_hidden, accuracy_fourth_hidden, accuracy_ffnn

In [ ]:
#acc = get_accuracy_values_at_noise_level(0.6);
#print("Accuracy of H1 read-out: %.3f" % acc[0])
#print("Accuracy of H2 read-out: %.3f" % acc[1])
#print("Accuracy of H3 read-out: %.3f" % acc[2])
#print("Accuracy of FF network : %.3f" % acc[3])

In [ ]:
def plot_noise_robustness_curves(noise_levels):
  accuracy_values_first_hidden = []
  accuracy_values_second_hidden = []
  accuracy_values_third_hidden = []
  accuracy_values_fourth_hidden = []
  accuracy_values_ffnn = []

  for noise_level in noise_levels:
    acc = get_accuracy_values_at_noise_level(noise_level)
    accuracy_values_first_hidden.append(acc[0])
    accuracy_values_second_hidden.append(acc[1])
    accuracy_values_third_hidden.append(acc[2])
    accuracy_values_fourth_hidden.append(acc[3])
    accuracy_values_ffnn.append(acc[4])

  fig, ax = plt.subplots()
  ax.plot(range(len(noise_levels)), accuracy_values_first_hidden)
  ax.plot(range(len(noise_levels)), accuracy_values_second_hidden)
  ax.plot(range(len(noise_levels)), accuracy_values_third_hidden)
  ax.plot(range(len(noise_levels)), accuracy_values_fourth_hidden)
  ax.plot(range(len(noise_levels)), accuracy_values_ffnn)

  ax.set_title("Robustness to noise")
  ax.set_xlabel("Noise level (%)")
  ax.set_ylabel("Accuracy")
  plt.xticks(range(len(noise_levels)), [int(l*100) for l in noise_levels])
  plt.legend(["First hidden", "Second hidden", "Third hidden", "Fourth hidden" ,"FFNN"])

In [ ]:
noise_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]
plot_noise_robustness_curves(noise_levels)

Adversarial attacks

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()

    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad

    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    # Return the perturbed image
    return perturbed_image

In [ ]:
class DBNWithReadOut(torch.nn.Module):
    def __init__(self, dbn_mnist, readouts, readout_level=0):
        super().__init__()
        self.readouts = readouts
        self.dbn_mnist = dbn_mnist
        self._require_grad()
        self.readout_level = readout_level

    def _require_grad(self):
      for rbm in self.dbn_mnist.rbm_layers:
        rbm.W.requires_grad_()
        rbm.h_bias.requires_grad_()

    def forward(self, image):
      """This forward pass uses probabilities instead of samples as RBM outputs
       to backpropagate the gradient"""
      p_v = image
      hidden_states = []
      for rbm in self.dbn_mnist.rbm_layers:
        p_v = p_v.view((p_v.shape[0], -1))  # flatten
        p_v, v = rbm(p_v)
        hidden_states.append(p_v)
      return self.readouts[self.readout_level].forward(hidden_states[self.readout_level])

In [ ]:
dbn_with_readout = DBNWithReadOut(dbn_emnist, [linear1, linear2, linear3], readout_level=2)

In [ ]:
test_sample_idx = 1
test_image = emnist_test.data[test_sample_idx].reshape(1, 784)
__ = plt.imshow(test_image.reshape(28,28).to('cpu'))

In [ ]:
attacked_model = ffnn

In [ ]:
attacked_model = dbn_with_readout

In [ ]:
test_image.requires_grad_()
model_outputs = attacked_model(test_image)
prediction = torch.argmax(model_outputs)
print(f"The prediction of the model for this clean sample is {prediction}.")

In [ ]:
epsilon = 0.2  # define strenght of the attack
test_image_label = emnist_test.targets[test_sample_idx].unsqueeze(0)  # get ground truth label for that image
loss_value = torch.nn.functional.cross_entropy(model_outputs, test_image_label)  # get loss value
attacked_model.zero_grad()
loss_value.backward()
image_grad = test_image.grad.data  # get the gradient of the pixels w.r.t. the loss
perturbed_image = fgsm_attack(test_image, epsilon, image_grad)

perturbed_image_np = perturbed_image.detach().to('cpu').numpy()
__ = plt.imshow(perturbed_image_np.reshape(28,28))

In [ ]:
model_outputs = attacked_model(perturbed_image.view((perturbed_image.shape[0], -1)))
print(f"The prediction of the model for the perturbed sample is {torch.argmax(model_outputs)}.")

In [ ]:
def test_robustness_to_attack(model, device, test_loader, epsilon, num_steps=0, verbose=True):
    correct = 0
    print_reconstruction = num_steps > 0

    for data, target in tqdm(test_loader):
        data, target = data.to(device), target.to(device)
        data = data.reshape(-1, 784)
        data.requires_grad = True  # Important for Attack

        output = model.forward(data)

        init_pred = torch.argmax(output)

        if (print_reconstruction and verbose):
          print("\nHere's the original sample:\n")
          plt.imshow(data[0].detach().to('cpu').numpy().reshape(28,28))
          plt.show()

        loss = functional.nll_loss(output, target)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data  # collect the gradient of the input data
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        if (print_reconstruction and verbose):
            print("\nHere's a perturbed sample:\n")
            plt.imshow(perturbed_data[0].detach().to('cpu').numpy().reshape(28,28))
            plt.show()


        # If requested, reconstruct the input iterating forward-backward dynamics
        if num_steps > 0:
            for __ in range(0, num_steps):
                perturbed_data, __ = model.dbn_mnist.reconstruct(perturbed_data)
            if (print_reconstruction and verbose):
                print(f"\nHere's what a {num_steps}-steps reconstructed sample looks like:\n")
                plt.imshow(perturbed_data[0].detach().to('cpu').numpy().reshape(28,28))
                plt.show()
                print_reconstruction = False

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        # get the index of the max element in the output
        final_pred = output.max(1, keepdim=True)[1]
        final_pred = output.argmax(-1)
        correct += (final_pred == target).sum()

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader.sampler))
    print("\nEpsilon: {}\nTest Accuracy: {:.2f}%\n".format(epsilon, final_acc*100))

    return final_acc.item()

In [ ]:
test_loader = torch.utils.data.DataLoader(
    tv.datasets.MNIST('data/', train=False, download=False, transform=tv.transforms.Compose([tv.transforms.ToTensor()])),
    batch_size=100, shuffle=True)

In [ ]:
final_acc = test_robustness_to_attack(ffnn, device,
                                      test_loader, epsilon=0.1,
                                      num_steps=0

In [ ]:
final_acc = test_robustness_to_attack(dbn_with_readout, device,
                                      test_loader, epsilon=0.1,
                                      num_steps=0)

In [ ]:
### TASK: do the same thing for the DBN with one top-down reconstruction step
final_acc = test_robustness_to_attack(dbn_with_readout, device,
                                      test_loader, epsilon=0.1,
                                      num_steps=1)

In [ ]:
epsilon_values = [0, 0.05, 0.10, 0.15, 0.20, 0.25]

def test_epsilon_values_effect(model, n_steps):
  accuracies = list()

  for eps in epsilon_values:
      acc = test_robustness_to_attack(model, device, test_loader, eps, num_steps=n_steps, verbose=False)
      accuracies.append(acc)

  return accuracies

In [ ]:
%%capture
accuracies_ffnn = test_epsilon_values_effect(ffnn, n_steps=0)
accuracies_dbn_0 = test_epsilon_values_effect(dbn_with_readout, n_steps=0)
accuracies_dbn_1 = test_epsilon_values_effect(dbn_with_readout, n_steps=1)
accuracies_dbn_2 = test_epsilon_values_effect(dbn_with_readout, n_steps=2)
accuracies_dbn_3 = test_epsilon_values_effect(dbn_with_readout, n_steps=3)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 7), sharey=True)

__ = ax.axhline(0.1, color='gray', linestyle=':')
__ = ax.plot(epsilon_values, accuracies_ffnn)
__ = ax.plot(epsilon_values, accuracies_dbn_0)
__ = ax.plot(epsilon_values, accuracies_dbn_1)
__ = ax.plot(epsilon_values, accuracies_dbn_2)
__ = ax.plot(epsilon_values, accuracies_dbn_3)
__ = ax.set_xlabel("Strength of adversarial attack ($\epsilon$)")
__ = ax.set_ylabel("Accuracy")
__ = ax.set_title("Robustness to adversarial attacks", {'fontsize': 15})
__ = ax.legend(["Chance level", "FFNN", "DBN", "DBN top-down","DBN top-down x2","DBN top-downx3"])